In [25]:
import requests
import json
import urllib
import pandas as pd
# import mysql.connector
from sqlalchemy import create_engine
import time
import logging
import subprocess
import pprint
import os
pp = pprint.PrettyPrinter(indent=4)
import pickle


In [5]:
url = '''https://api.twitch.tv/helix/games/top?first=100'''

In [ ]:
def get_top_100_games():
    # Need to pass client ID with each request in header
    headers = {'Client-ID': 'vb2kmh60pt0tee6o2c11ko6n2t1w9a'}
    url = '''https://api.twitch.tv/helix/games/top?first=100'''
    r = requests.get(url, headers=headers)
    return r

r_dict = json.loads(r.text)

In [6]:
headers = {'Client-ID': 'vb2kmh60pt0tee6o2c11ko6n2t1w9a'}

r = requests.get(url, headers=headers)

In [55]:
req.headers['Ratelimit-Remaining']

'29'

In [8]:
r_dict = json.loads(r.text)

In [13]:
r_dict['data'][0]['id']

'33214'

In [88]:
time.strftime("%Y-%m-%d %H:%M:%S",time.gmtime())


'2019-05-31 02:15:28'

In [ ]:
def 

In [63]:
def check_api_limit_reached(req):
    if int(req.headers['Ratelimit-Remaining']) <= 1: # No more requests, need to pause for 30s
        print('Waiting for API limit to refresh (30s)')
        time.sleep(30)
        print('Continuing')
    return req.headers['Ratelimit-Remaining']

In [115]:
stream_dict = dict()
url = 'https://api.twitch.tv/helix/streams?first=100&gameid='
for game in r_dict['data']:
    req = requests.get(url + game['id'],headers=headers)
    check_api_limit_reached(req)    
    stream_dict[game['name']]=json.loads(req.text)


Waiting for API limit to refresh (30s)
Continuing
Waiting for API limit to refresh (30s)
Continuing
Waiting for API limit to refresh (30s)
Continuing


In [121]:
with open("data_file.json", "w") as write_file:
    json.dump(stream_dict, write_file)

In [122]:
del stream_dict

In [123]:
with open("data_file.json", "r") as read_file:
    data = json.load(read_file)

In [117]:
whos

Variable                  Type             Data/Info
----------------------------------------------------
check_api_limit           function         <function check_api_limit at 0x11eb45b70>
check_api_limit_reached   function         <function check_api_limit_reached at 0x120d68f28>
create_engine             function         <function create_engine at 0x11b2c68c8>
game                      dict             n=3
headers                   dict             n=1
json                      module           <module 'json' from '/ana<...>hon3.7/json/__init__.py'>
logging                   module           <module 'logging' from '/<...>3.7/logging/__init__.py'>
os                        module           <module 'os' from '/anaco<...>da3/lib/python3.7/os.py'>
output_list               list             n=99
pd                        module           <module 'pandas' from '/a<...>ages/pandas/__init__.py'>
pickle                    module           <module 'pickle' from '/a<...>lib/python3.7/pickle.p

In [26]:
 pickle.dump(output_list, open( "output.p", "wb" ) )

In [28]:
!pwd

/Users/jchow/Work/MetisCode/twitch-streamer-recommender/gitRoot/Twitch_Streamers_Recommender/Scraper_Notebook


In [34]:
tfue_id = output_list[0]['data'][0]['user_id']
#[0]['user_id']

In [96]:
json.loads(testrequest)

{'total': 6166637,
 'data': [{'from_id': '438578472',
   'from_name': 'hentayuu',
   'to_id': '60056333',
   'to_name': 'Tfue',
   'followed_at': '2019-05-31T02:15:32Z'}],
 'pagination': {'cursor': 'eyJiIjpudWxsLCJhIjp7IkN1cnNvciI6IjE1NTkyNjg5MzIyMzk5MDQzNTQifX0'}}

In [90]:
url = 'https://api.twitch.tv/helix/users/follows?first=1&to_id=' + tfue_id
testrequest=requests.get(url,headers=headers).text

In [39]:
# Example api ping through command line
# !curl -H 'Client-ID: vb2kmh60pt0tee6o2c11ko6n2t1w9a' \-X GET https://api.twitch.tv/helix/games/top